<a href="https://colab.research.google.com/github/ibsenvillarroel/iris-flowers-classification/blob/main/Iris_flower_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%tensorflow_version 2.x

In [2]:
from __future__ import absolute_import, division, print_function, unicode_literals

import tensorflow as tf

import pandas as pd

Dataset

This specific dataset separates flowers into 3 different classes of species 



*  Setosa
*  Versicolor
*  Virginica

The information about this flowers 
* sepal length 
* sepal width
* petal length 
* petal width  

In [3]:
CSV_COLUM_NAMES = ["SepalLength", "SepalWidth", "PetalLength", "PetalWidth", "Species"]
SPECIES= ["Setosa", "Versicolor", "Virginica"]

#lest define some constants / definimos algunas variables

In [4]:
train_path = tf.keras.utils.get_file(
    "iris_training.csv", "https://storage.googleapis.com/download.tensorflow.org/data/iris_training.csv")
test_path = tf.keras.utils.get_file(
    "iris_test.csv", "https://storage.googleapis.com/download.tensorflow.org/data/iris_test.csv")

train = pd.read_csv(train_path, names=CSV_COLUM_NAMES, header=0) 
test = pd.read_csv(test_path, names=CSV_COLUM_NAMES, header=0)

#here we use keras (a module inside of tensorflow) to grab our dtatasets and read them into a pandas dataframe 

16384/573 [=========================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================] - 0s 0us/step


In [5]:
train.head()

,SepalLength,SepalWidth,PetalLength,PetalWidth,Species
0,6.4,2.8,5.6,2.2,2
1,5.0,2.3,3.3,1.0,1
2,4.9,2.5,4.5,1.7,2
3,4.9,3.1,1.5,0.1,0
4,5.7,3.8,1.7,0.3,0


In [6]:
train_y = train.pop("Species")
test_y = test.pop("Species")
train.head()

,SepalLength,SepalWidth,PetalLength,PetalWidth
0,6.4,2.8,5.6,2.2
1,5.0,2.3,3.3,1.0
2,4.9,2.5,4.5,1.7
3,4.9,3.1,1.5,0.1
4,5.7,3.8,1.7,0.3


In [7]:
train.shape

(120, 4)

In [18]:
def input_fn(features, labels, training=True, batch_size=256):
  #convert the inputs to a dataset
  dataset = tf.data.Dataset.from_tensor_slices((dict(features), labels))

  #shuffle and repeat if you are in a training mode
  if training:
    dataset= dataset.shuffle(1000).repeat()

  return dataset.batch(batch_size)   

In [14]:
my_feature_columns = []
for key in train.keys():
  my_feature_columns.append(tf.feature_column.numeric_column(key=key))
print(my_feature_columns)

[NumericColumn(key='SepalLength', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='SepalWidth', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='PetalLength', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='PetalWidth', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None)]


building the model 

Utilizaremos un modelo de neural network, DNNClassifier

In [15]:
classifier = tf.estimator.DNNClassifier(
    feature_columns= my_feature_columns,
    #two hidden layers of 30 and 10 nodes respectively
    hidden_units=[30, 10], 
    #the model must chose between 3 classes
    n_classes=3)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/tmp/tmp1dcijrwz', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_checkpoint_save_graph_def': True, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


Training the model 

In [19]:
classifier.train(
    input_fn= lambda: input_fn(train, train_y, training=True),
    steps = 5000) #lambda lo q hace es devolvernos una funcion como un objeto funcion 

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Calling checkpoint listeners before saving checkpoint 0...
INFO:tensorflow:Saving checkpoints for 0 into /tmp/tmp1dcijrwz/model.ckpt.
INFO:tensorflow:Calling checkpoint listeners after saving checkpoint 0...
INFO:tensorflow:loss = 1.0044935, step = 0
INFO:tensorflow:global_step/sec: 400.108
INFO:tensorflow:loss = 0.9117144, step = 100 (0.255 sec)
INFO:tensorflow:global_step/sec: 509.451
INFO:tensorflow:loss = 0.86437607, step = 200 (0.197 sec)
INFO:tensorflow:global_step/sec: 467.625
INFO:tensorflow:loss = 0.8296948, step = 300 (0.212 sec)
INFO:tensorflow:global_step/sec: 509.071
INFO:tensorflow:loss = 0.73262596, step = 400 (0.198 sec)
INFO:tensorflow:global_step/sec: 501.676
INFO:tensorflow:loss = 0.68743813, step = 500 (0.198 sec)
INF

In [20]:
eval_result = classifier.evaluate(input_fn= lambda: input_fn(test, test_y, training=False))
print('\nTest set accuracy: {accuracy:0.3f}\n'.format(**eval_result))

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2022-04-19T03:51:44
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmp1dcijrwz/model.ckpt-5000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Inference Time : 0.31971s
INFO:tensorflow:Finished evaluation at 2022-04-19-03:51:45
INFO:tensorflow:Saving dict for global step 5000: accuracy = 0.96666664, average_loss = 0.41205865, global_step = 5000, loss = 0.41205865
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 5000: /tmp/tmp1dcijrwz/model.ckpt-5000

Test set accuracy: 0.967



Predictions

Hacemos una funcion que nos pide valores de entrada de una flor y segun eso predecimos que tipo de flor es 

In [23]:
def input_fn(features, batch_size=256):
    # Convert the inputs to a Dataset without labels.
    return tf.data.Dataset.from_tensor_slices(dict(features)).batch(batch_size)

features = ['SepalLength', 'SepalWidth', 'PetalLength', 'PetalWidth']
predict = {}

print("Please type numeric values as prompted.")
for feature in features:
  valid = True
  while valid: 
    val = input(feature + ": ")
    if not val.isdigit(): valid = False

  predict[feature] = [float(val)]

predictions = classifier.predict(input_fn=lambda: input_fn(predict))
for pred_dict in predictions:
    print(pred_dict) #con esto veo todas las probabilidades
    class_id = pred_dict['class_ids'][0]
    probability = pred_dict['probabilities'][class_id]

    print('Prediction is "{}" ({:.1f}%)'.format(
        SPECIES[class_id], 100 * probability))


Please type numeric values as prompted.
SepalLength: 0.1
SepalWidth: 0.2
PetalLength: 0.3
PetalWidth: 0.4
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmp1dcijrwz/model.ckpt-5000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
{'logits': array([ 0.02987338,  0.17827281, -0.20198925], dtype=float32), 'probabilities': array([0.3386351 , 0.39280862, 0.26855627], dtype=float32), 'class_ids': array([1]), 'classes': array([b'1'], dtype=object), 'all_class_ids': array([0, 1, 2], dtype=int32), 'all_classes': array([b'0', b'1', b'2'], dtype=object)}
Prediction is "Versicolor" (39.3%)
